<a href="https://colab.research.google.com/github/AlecTraas/computational-geo-lab/blob/main/Colab/Kai/quickhull_algo_3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random as rnd

In [ ]:
def main(num_points):
  points = sorted([[rnd.random()*10, rnd.random()*10, rnd.random()*10] for _ in range(num_points)], key=lambda x: x[0])
  np_points = np.array(points)
  fig = plt.figure()
  ax = fig.add_subplot(projection='3d')
  ax.scatter(np_points[:,0], np_points[:,1], np_points[:,2])
  hull = ConvexHull(np_points)
  hull.quick_hull()
  np_hull_pts = np.array(hull.hull_points)
  np_hull_fcts = np.array(hull.hull_facets)
  ax.plot_trisurf(np_hull_pts[:, 0], np_hull_pts[:, 1], np_hull_pts[:, 2], triangles=np_hull_fcts, color='red', alpha=0.5)
  plt.show()

In [ ]:
# I miss structs :(
class ConvexHull():
  def __init__(self, points):
    # [[x,y,z]...]
    self.points = points
    # [[x,y,z]...]
    self.hull_points = None
    # references indecies in the hull_points array
    # [[a,b,c]...]
    self.hull_facets = None
    # references indecies in the hull_points array
    # [[a,b]...]
    self.hull_ridges = None
    # references indecies in the hull_ridges array
    # [a,b,c...]
    self.neighbors = None

    self.above_tri_sets = None

  def quick_hull(self):
    # I assume, for the sake of simplicity, that no one will input fewer that 4 points
    self.define_tetrahedron()
    # ↓ I'm going to use Lam's method!
    self.clear_internal_points()

  def define_tetrahedron(self):
    temp = np.array(self.points)
    # ↓ Inefficient, but it works
    i = []
    k = temp[:,0].argmin()
    i.append(k)
    temp[k] = np.inf
    k = temp[:,1].argmin()
    i.append(k)
    temp[k] = np.inf
    k = temp[:,2].argmin()
    i.append(k)
    temp[temp == np.inf] = -np.inf
    temp[k] = -np.inf
    k = temp[:,1].argmax()
    i.append(k)
    i = sorted(i, reverse=True)
    self.hull_points = [self.points[j, :] for j in i]
    for j in i:
      self.points = np.delete(self.points,j,0)
    self.hull_points.sort(key=lambda x: x[0])
    self.hull_points = self.hull_points[::-1]
    self.hull_facets = [[0,2,1],[0,1,3],[0,3,2],[1,2,3]]
    self.neighbors = [[[0,3,2],[1,2,3],[0,1,3]],
                      [[0,2,1],[1,2,3],[0,3,2]],
                      [[0,1,3],[1,2,3],[0,2,1]],
                      [[0,2,1],[0,3,2],[0,1,3]]]

  def above_facet(self,face,point):
    face = [self.hull_points[face[0]],self.hull_points[face[1]],self.hull_points[face[2]]]
    n = np.cross((face[1]-face[0]),(face[2]-face[0]))
    n = n/np.linalg.norm(n)
    d = np.dot(n,(face[0]-point))
    return d > 0

  def clear_internal_points(self):
    hull_centroid = np.sum(self.hull_points, axis=0)/len(self.hull_points)
    normals = [[]]
    for f in self.hull_facets:
      normals.append(face_to_point_list(f)/3)
    p_queue = [[]]
    for p in self.points:
      for n in normals:
        if np.dot(p-n,n-hull_centroid) > 0:
          p_queue.append(p)
    # clear duplicates in the p_queue list
    self.points.clear(p_queue)

  def face_to_point_list(self,f): # still working on this
    p = self.hull_points
    return [[p[f[0,:]]]]

  def furthest_point(self,face):
    """
    furthest = 0
    for i,p in enumerate(above_tri_sets[face_index]): # is enumerate the way to get index and point? not sure
      if dist(p,face) > furthest:
        furthest = i
    return furthest
    """

  def init_above_tri_sets(self):
    """
    I came up with my own method, no wikipedia or anything!! :))
    - find the average point for each face of the tet (a+b+c)/3
    - assign point to face which has min dist between average and the point

    Something like:

    centers = [avg for avg in average(facets)]
    for p in points:
      above_tri_sets[min_dist_index(p,centers)].append(p)
    """

  def join_point_to_horizon(self,horizon,point):
    """
    - for ridge in horizon:
      - new facet (point,ridge.a,rigde.b)
    - delete_internal_points()
    """

In [ ]:
main(1000)